In [302]:
# Чтобы скачать и распаковать zip-архив в output папку в Питоне:
import pandas as pd
import numpy as np
from IPython.display import display
#!/usr/bin/env python
import io
import zipfile
import requests  # $ pip install requests

r = requests.get("https://lms.skillfactory.ru/assets/courseware/v1/8e23b4ad7fb8d49f0b9c67be84dc440c/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/ratings_movies.zip")
with r, zipfile.ZipFile(io.BytesIO(r.content)) as archive:
    archive.extractall('Data')

In [303]:
df = pd.read_csv('Data/ratings_movies.csv', sep=',')
df = df.drop('Unnamed: 0', axis=1)
display(df.head())
df['date'] = pd.to_datetime(df['date'])
# df['userId'] = df['userId'].astype('category')
# df['genres'] = df['genres'].astype('category')
df.info()


,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   userId   100836 non-null  int64         
 1   movieId  100836 non-null  int64         
 2   rating   100836 non-null  float64       
 3   date     100836 non-null  datetime64[ns]
 4   title    100836 non-null  object        
 5   genres   100836 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 4.6+ MB


In [304]:
# Для решения задач нам понадобится выделить из признака title год выпуска фильма. Для этого напишем функцию get_year_release(arg).
# библиотека для регулярных выражений

import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return 0
 
# Задание 8.1
# Создайте в таблице новый признак year_release, который соответствует году выпуска фильма.
# У скольких фильмов не указан год их выпуска? 
    
df['year_release'] = df['title'].apply(get_year_release)
mask_0 = df['year_release'] == 0 
display(df[mask_0]['movieId'].count())
df.info()


18

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   userId        100836 non-null  int64         
 1   movieId       100836 non-null  int64         
 2   rating        100836 non-null  float64       
 3   date          100836 non-null  datetime64[ns]
 4   title         100836 non-null  object        
 5   genres        100836 non-null  object        
 6   year_release  100836 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(2)
memory usage: 5.4+ MB


In [305]:
# Задание 8.2
# Какой фильм, выпущенный в 1999 году, получил наименьшую среднюю оценку зрителей?
# В качестве ответа запишите название этого фильма без указания года его выпуска.

mask_year_1999 = df['year_release'] == 1999
df_1999 =df[mask_year_1999]
type(df_1999.head())
df_grouped = df_1999.groupby(by=['movieId', 'title'], as_index=False)['rating'].mean().sort_values(by='rating').head(1)['title'].values
display(str(*df_grouped).replace(' (1999)', ''))

'Bloodsport: The Dark Kumite'

In [306]:
# Задание 8.3
# 0/1 point (graded)
# Какое сочетание жанров фильмов (genres), выпущенных в 2010 году, получило наименьшую среднюю оценку (rating)?
# Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов).

mask_year_2010 = df['year_release'] == 2010
df_2010 =df[mask_year_2010]
type(df_2010.head())
df_grouped = df_2010.groupby(by=['genres'], as_index=False)['rating'].mean().sort_values(by='rating').head(1)['genres'].values
display(*df_grouped)

'Action|Sci-Fi'

In [307]:
# Задание 8.4
# Какой пользователь (userId) посмотрел наибольшее количество различных (уникальных) жанров (genres) фильмов? 
# В качестве ответа запишите идентификатор этого пользователя.

df.groupby('userId')['genres'].nunique().idxmax()
# display(df[df['userId'] == 599].info())

599

In [308]:
# Задание 8.5
# Найдите пользователя, который выставил наименьшее количество оценок, но его средняя оценка фильмам наибольшая.
# В качестве ответа укажите идентификатор этого пользователя.
# Чтобы рассчитать несколько параметров для каждого пользователя (количество оценок и среднюю оценку), можно воспользоваться методом agg() на сгруппированных данных.

df.groupby('userId')['rating'].agg(['count', 'mean']).sort_values(by= ['mean', 'count'], ascending=[False, True]).idxmax()['mean']

53

In [309]:
# Задание 8.6
# Найдите сочетание жанров (genres) за 2018 году, которое имеет наибольший средний рейтинг (среднее по столбцу rating), 
# и при этом число выставленных ему оценок (количество значений в столбце rating) больше 10.
# Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов).

mask_year_2018 = df['year_release'] == 2018
# mask_count_rating = df.groupby(['genres'])['rating'].count() > 10
df_main = df[mask_year_2018].groupby('genres', as_index=True)['rating'].agg(['mean', 'count']).sort_values(by='count', ascending=False)
mask_count = df_main['count'] >10
df_main[mask_count].idxmax()['mean']

'Action|Adventure|Sci-Fi'

In [369]:
# Задание 8.7
# Добавьте в таблицу новый признак year_rating — год выставления оценки. 
df['year_rating'] = df['date'].dt.year.astype('category')
# df.info()
# Создайте сводную таблицу, которая иллюстрирует зависимость среднего рейтинга фильма от года выставления оценки и жанра.
table = df.pivot_table(values='rating', index='genres', columns='year_rating', fill_value= 0, aggfunc='mean').round(2)

# Выберите верные варианты ответа, исходя из построенной таблицы:

# A За весь период (с 1996 по 2018 год) сочетание жанров Action|Adventure ни разу не получало среднюю оценку ниже 3.
# вывод части сводной таблицы
table.query('genres== ["Action|Adventure"]')
#False
# проверка минимального значения по жанру
df[df['genres'] == "Action|Adventure"].groupby(['genres', 'year_rating'], as_index=True)['rating'].mean().min()

# B Наилучшую оценку жанр Action|Adventure|Animation|Children|Comedy|IMAX получил в 2010 году.(2018)
max(table.query('genres== ["Action|Adventure|Animation|Children|Comedy|IMAX"]')) == 2010
#False

# C Среди сочетаний жанров, получивших наивысшую среднюю оценку в 2018 году, есть сочетание Animation|Children|Mystery.

df[df['year_rating'] == 2018].groupby('genres', as_index=True)['rating'].mean().sort_values(ascending= False).nlargest(20)

# D Для жанра Comedy прослеживается тенденция падения рейтинга с каждым годом (с 1996 по 2018).
table.query('genres== ["Comedy"]')


year_rating,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
genres,,,,,,,,,,,,,,,,,,,,,
Comedy,3.23,3.41,3.0,3.61,3.14,3.32,3.2,3.12,3.36,2.96,...,3.27,3.18,3.23,3.48,3.49,3.27,3.11,3.37,2.85,3.12


In [371]:
# Чтобы скачать и распаковать zip-архив в output папку в Питоне:
import pandas as pd
import numpy as np
from IPython.display import display
#!/usr/bin/env python
import io
import zipfile
import requests  # $ pip install requests

r = requests.get("https://lms.skillfactory.ru/assets/courseware/v1/a4cb93123b3b6998d57348053023f4e0/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/orders_and_products.zip")
with r, zipfile.ZipFile(io.BytesIO(r.content)) as archive:
    archive.extractall('Data')

# https://lms.skillfactory.ru/assets/courseware/v1/a4cb93123b3b6998d57348053023f4e0/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/orders_and_products.zip

In [382]:
orders = pd.read_csv('Data/orders.csv', sep=';')
# df = df.drop('Unnamed: 0', axis=1)
display(orders.head())
display(orders.info())
products = pd.read_csv('Data/products.csv', sep=';')
# df = df.drop('Unnamed: 0', axis=1)
display(products.head())
display(products.info())

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5
1,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100
2,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10
3,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7
4,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Дата создания  18 non-null     object
 1   Order ID       18 non-null     int64 
 2   ID Покупателя  18 non-null     int64 
 3   Статус         18 non-null     object
 4   Оплачен        18 non-null     object
 5   Отменен        18 non-null     object
 6   Отгружен       18 non-null     object
 7   ID товара      18 non-null     int64 
 8   Количество     18 non-null     int64 
dtypes: int64(4), object(5)
memory usage: 1.4+ KB


None

,Product_ID,Name,Price,CURRENCY
0,47,Шатны Полосатый рейс,2999,RUR
1,51,Платье Аленький цветочек,4999,RUR
2,53,Штаны Цветочная Поляна,4999,RUR
3,71,Платье Ночная Жизнь,7999,RUR
4,74,Платье Ночная Жизнь XXXL,8999,RUR


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Product_ID  14 non-null     int64 
 1   Name        14 non-null     object
 2   Price       14 non-null     int64 
 3   CURRENCY    14 non-null     object
dtypes: int64(2), object(2)
memory usage: 576.0+ bytes


None

In [392]:
ord_prod = orders.merge(products, how='left', left_on='ID товара', right_on='Product_ID')
display(ord_prod.head())
display(ord_prod.info())


,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество,Product_ID,Name,Price,CURRENCY
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,103.0,"Носки Подарочные, муж",199.0,RUR
1,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100,86.0,"Носки Простые, муж",45.0,RUR
2,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10,104.0,"Носки Подарочные, жен",249.0,RUR
3,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7,104.0,"Носки Подарочные, жен",249.0,RUR
4,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5,104.0,"Носки Подарочные, жен",249.0,RUR


<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 0 to 17
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Дата создания  18 non-null     object 
 1   Order ID       18 non-null     int64  
 2   ID Покупателя  18 non-null     int64  
 3   Статус         18 non-null     object 
 4   Оплачен        18 non-null     object 
 5   Отменен        18 non-null     object 
 6   Отгружен       18 non-null     object 
 7   ID товара      18 non-null     int64  
 8   Количество     18 non-null     int64  
 9   Product_ID     17 non-null     float64
 10  Name           17 non-null     object 
 11  Price          17 non-null     float64
 12  CURRENCY       17 non-null     object 
dtypes: float64(2), int64(4), object(7)
memory usage: 2.0+ KB


None

In [419]:
# Задание 8.8
# Какой идентификатор (Order ID) имеет заказ, для которого не оказалось информации о товаре?
ord_prod = ord_prod.fillna(0)
# ord_prod.info()
mask_Name =ord_prod['Name'] == 0
display(ord_prod[mask_Name]['Order ID'])

17    0
Name: Order ID, dtype: int64

In [425]:
# Задание 8.9
# На какой товар была произведена отмена?
# В качестве ответа запишите название этого товара (Name).

z = ord_prod[ord_prod['Отменен'] == 'Да']['Name'].values
print(*z)

Носки беговые Camino


In [428]:
# Задание 8.10
# Какой покупатель принёс наибольшую суммарную прибыль интернет-магазину за указанный период?
# В ответ запишите идентификатор этого покупателя (ID Покупателя).
# Прибыль состоит только из оплаченных заказов и рассчитывается как количество купленного товара, умноженное на его цену.

ord_prod['Profit'] = ord_prod['Price'] * ord_prod['Количество']
mask_payed = ord_prod['Оплачен'] == 'Да'
ord_prod[mask_payed].groupby('ID Покупателя', as_index=True)['Profit'].sum().idxmax()



7